In [11]:
#import all packages
import pandas as pd
import numpy as np
import re as re
import nltk
import string, unicodedata
from nltk.stem.porter import *

from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from datetime import datetime, timedelta

In [12]:
def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

In [13]:
# datafile
data_file = "AllData_Raw.csv"
data = pd.read_csv(data_file)
data.columns = data.columns.str.strip().str.replace(" ","_") # reason for doing this is cause cannot get columns with space into a list

#reformat the respective columns

#lower case
data['Review'] = [word.lower() for word in data['Review']]

#Strip off the Variation Word
data['Product_Purchase'] = data['Product_Purchase'].replace('Variation', '')

#Change Date of Review to DateTime
#data['Date_Of_Review'] = pd.to_datetime(data['Date_Of_Review'])

data.info()
data.head()
#data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9002 entries, 0 to 9001
Data columns (total 11 columns):
Platform            9002 non-null object
Brand               9002 non-null object
Category            9002 non-null object
Product_Name        9002 non-null object
Price               9002 non-null float64
Reviewer            8980 non-null object
Review              9002 non-null object
Product_Purchase    7394 non-null object
Ratings             8980 non-null float64
Date_Of_Review      8980 non-null object
Response            8980 non-null object
dtypes: float64(2), object(9)
memory usage: 773.7+ KB


,Platform,Brand,Category,Product_Name,Price,Reviewer,Review,Product_Purchase,Ratings,Date_Of_Review,Response
0,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,zenheng,"delivery was quite fast, and item was on disco...",NaN,4.0,2020-02-06 23:58,no
1,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,glennoeestaquio,received item sealed properly.\r\r\nquick deli...,NaN,5.0,2020-02-06 12:44,no
2,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,shy2206,yet to try but delivery was prompt and product...,NaN,5.0,2020-02-06 10:21,no
3,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,lindamiyalin,"well wrapped, in good condition, super fast ...",NaN,5.0,2020-01-30 23:01,no
4,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,kyc385,"well packed, will buy again",NaN,5.0,2020-02-01 11:40,no


### Transformation and Initalise of data

In [14]:
# make the old csv data into a list

platform_list = data['Platform'].tolist()
brand_list = data['Brand'].tolist()
category_list = data['Category'].tolist()
product_name_list = data['Product_Name'].tolist()
price_list = data['Price'].tolist()
reviewer_list = data['Reviewer'].tolist()
review_list = data['Review'].tolist()
product_variation_list = data['Product_Purchase'].tolist()
rating_list = data['Ratings'].tolist()
date_review_list = data["Date_Of_Review"].tolist()
response_list = data["Response"].tolist()

# initialising the new columns

platform_csv=[]
brand_csv = []
category_csv = []
product_name_csv = []
prices_csv = []
reviewer_csv =[]
review_splitted_csv = []
review_csv=[]
product_variation_csv = []
rating_csv = []
date_review_csv = []
response_csv = []

# this is the length of the csv old data
print ("The length of the dataset is:", len(brand_list))

The length of the dataset is: 9002


In [15]:
#upload the appos
from word_dict import appos

print(appos)

{"aren't": 'are not', "can't": 'cannot', "couldn't": 'could not', "didn't": 'did not', "doesn't": 'does not', "don't": 'do not', "hadn't": 'had not', "hasn't": 'has not', "haven't": 'have not', "he'd": 'he would', "he'll": 'he will', "he's": 'he is', "i'd": 'I had', "i'll": 'I will', "i'm": 'I am', "isn't": 'is not', "it's": 'it is', "it'll": 'it will', "i've": 'I have', "let's": 'let us', "l'oreal": 'loreal', "mightn't": 'might not', "mustn't": 'must not', "shan't": 'shall not', "she'd": 'she would', "she'll": 'she will', "she's": 'she is', "shouldn't": 'should not', "that's": 'that is', "there's": 'there is', "they'd": 'they would', "they'll": 'they will', "they're": 'they are', "they've": 'they have', "we'd": 'we would', "we're": 'we are', "weren't": 'were not', "we've": 'we have', "what'll": 'what will', "what're": 'what are', "what's": 'what is', "what've": 'what have', "where's": 'where is', "who'd": 'who would', "who'll": 'who will', "who're": 'who are', "who's": 'who is', "who'

### Transform the Date Format

In [16]:
new_date =[]
for i in range(len(brand_list)):
    
    date_review = date_review_list[i]
    
    if platform_list[i] == 'Lazada':
        if 'days ago' in date_review or 'day ago' in date_review :
            day = date_review.replace('days ago', '').replace('day ago', '')
            date_review = datetime.today() - timedelta(days=int(day))
            date_review = date_review.strftime("%d-%b-%y")
#             print("trans",date_review)

        elif 'weeks ago' in date_review or 'week ago' in date_review:
            day = date_review.replace('weeks ago', '').replace('week ago', '')
            day = int(day)*7
            date_review = datetime.today() - timedelta(days=day)
            date_review = date_review.strftime("%d-%b-%y")
#             print("trans",date_review)

        elif 'hours ago' in date_review or 'hour ago' in date_review:
            hours = date_review.replace('hours ago', '').replace('hour ago', '')
            date_review = datetime.today() - timedelta(hours=int(hours))
            date_review = date_review.strftime("%d-%b-%y")
    
#         print("trans",date_review)
        new_date.append(date_review)
    else:
#         print(platform_list[i])
        new_date.append(date_review)
        
data["Date_Of_Review"] = new_date

new_date_review_list = data["Date_Of_Review"].tolist()

### Spilt the Sentence 

In [17]:
import itertools
from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize

spell = SpellChecker()

for i in range(len(brand_list)):
    # current_review is a string
    current_review = review_list[i]
    #print(current_review)
  
    processed_review = current_review.strip()
    print(processed_review)
    # Negation handling
    processed_review = processed_review.split()
    print("1",processed_review)
    processed_review =[appos[w] if w in appos else w for w in processed_review]
    processed_review = " ".join(processed_review) 
    
    print("2",processed_review)

    processed_review = processed_review.strip()
    # Remove all the special characters
    processed_review = re.sub(r'\W', ' ', processed_review)
    # Removing prefixed 'b'
    processed_review = re.sub(r'^b\s+', '', processed_review)
    # remove all single characters contains a white space character
    processed_review= re.sub(r'^[a-zA-Z]$', ' ', processed_review)
    # Substituting multiple spaces with single space
    processed_review = re.sub(r'\s+', ' ', processed_review, flags=re.I)


    processed = processed_review.strip()
   
    # final check if the processed_review is empty
    final = []
    if processed != "":
        tokenized = word_tokenize(processed)
        #removed additional letters eg "gooooooodddddd" to "good"
        for w in tokenized:
            processed_2 = reduce_lengthening(w)
            processed_2 = spell.correction(processed_2)
            final.append(processed_2)
        final =' '.join(final)
        print(final)
        # append into a dictionary
        platform_csv.append(platform_list[i])
        brand_csv.append(brand_list[i])
        category_csv.append(category_list[i])
        product_name_csv.append(product_name_list[i])
        prices_csv.append(price_list[i])
        reviewer_csv.append(reviewer_list[i])
        review_csv.append(current_review)
        review_splitted_csv.append(final)
        product_variation_csv.append(product_variation_list[i])
        rating_csv.append(rating_list[i])
        date_review_csv.append(new_date_review_list[i])
        response_csv.append(response_list[i])


delivery was quite fast, and item was on discount too so i couldn’t complain. so far liking this product! will continue to use it :)
1 ['delivery', 'was', 'quite', 'fast,', 'and', 'item', 'was', 'on', 'discount', 'too', 'so', 'i', 'couldn’t', 'complain.', 'so', 'far', 'liking', 'this', 'product!', 'will', 'continue', 'to', 'use', 'it', ':)']
2 delivery was quite fast, and item was on discount too so i couldn’t complain. so far liking this product! will continue to use it :)
delivery was quite fast and item was on discount too so i could t complain so far liking this product will continue to use it
received item sealed properly.
quick delivery too.
item not tested yet as for gifting purpose!
1 ['received', 'item', 'sealed', 'properly.', 'quick', 'delivery', 'too.', 'item', 'not', 'tested', 'yet', 'as', 'for', 'gifting', 'purpose!']
2 received item sealed properly. quick delivery too. item not tested yet as for gifting purpose!
received item sealed properly quick delivery too item not te

and round purchase after having quite good improvement from the first time usage of the essence high quality product and fitted well to the asian facial skins seller provided faster service and delivery thanks seller and hope
#shopeehaul good product quality fast delivery excellent service by seller
products were nicely packed and delivered. got a lot of free stuff too! thanks seller!
1 ['#shopeehaul', 'good', 'product', 'quality', 'fast', 'delivery', 'excellent', 'service', 'by', 'seller', 'products', 'were', 'nicely', 'packed', 'and', 'delivered.', 'got', 'a', 'lot', 'of', 'free', 'stuff', 'too!', 'thanks', 'seller!']
2 #shopeehaul good product quality fast delivery excellent service by seller products were nicely packed and delivered. got a lot of free stuff too! thanks seller!
shopeehaul good product quality fast delivery excellent service by seller products were nicely packed and delivered got a lot of free stuff too thanks seller
fast delivery. first time trying. will buy again i

received in good condition bought on sale masks were free tried on the mask and it felt hydration
received after 3days from order. hope the product doea what it claims
1 ['received', 'after', '3days', 'from', 'order.', 'hope', 'the', 'product', 'doea', 'what', 'it', 'claims']
2 received after 3days from order. hope the product doea what it claims
received after days from order hope the product does what it claims
received well as adv.good quality and i like it's smell. hopfully my skin will be better and glow. thanks loreal 😍
1 ['received', 'well', 'as', 'adv.good', 'quality', 'and', 'i', 'like', "it's", 'smell.', 'hopfully', 'my', 'skin', 'will', 'be', 'better', 'and', 'glow.', 'thanks', 'loreal', '😍']
2 received well as adv.good quality and i like it is smell. hopfully my skin will be better and glow. thanks loreal 😍
received well as adv good quality and i like it is smell hopefully my skin will be better and glow thanks lorean
speedy delivery. yet to try the product. hope it works o

ordered on sept received on sept yet to try meg date for 130ml crystal micro essence is jun 2019 exp date for uv perfect is may 2022 all in sealed bag
items received in good condition thank you
1 ['items', 'received', 'in', 'good', 'condition', 'thank', 'you']
2 items received in good condition thank you
items received in good condition thank you
product came quickly and incredibly well-packaged with a free gift :) thank you.
1 ['product', 'came', 'quickly', 'and', 'incredibly', 'well-packaged', 'with', 'a', 'free', 'gift', ':)', 'thank', 'you.']
2 product came quickly and incredibly well-packaged with a free gift :) thank you.
product came quickly and incredibly well packaged with a free gift thank you
good product quality, fast delivery
1 ['good', 'product', 'quality,', 'fast', 'delivery']
2 good product quality, fast delivery
good product quality fast delivery
fast delivery, received with 2weeks.. really worth the price, compare to shop is really much more cheaper and cone with the 

i received lorean revitalise crystal micro essence water in good condition and with free gift thank you
received about 2-3 days after placing order, together with the sunscreen. used for a few days, pimples seems less angry now. hopefully will be able to live up to the good reviews i read online.
1 ['received', 'about', '2-3', 'days', 'after', 'placing', 'order,', 'together', 'with', 'the', 'sunscreen.', 'used', 'for', 'a', 'few', 'days,', 'pimples', 'seems', 'less', 'angry', 'now.', 'hopefully', 'will', 'be', 'able', 'to', 'live', 'up', 'to', 'the', 'good', 'reviews', 'i', 'read', 'online.']
2 received about 2-3 days after placing order, together with the sunscreen. used for a few days, pimples seems less angry now. hopefully will be able to live up to the good reviews i read online.
received about 2 3 days after placing order together with the sunscreen used for a few days pimples seems less angry now hopefully will be able to live up to the good reviews i read online
fast delivery. 

...

okay so this is an interesting formula for a matte hippies i have tons of reason to love it it s quick drying and transferproof it s sticky before it s dries but its normal for a matte hippies it has a velvet felt when u first applied but has a silky felt after i even tried using lip balm over it and it s okay not only it s pigmented and has vibrant colour the one i received creator is a purple but has pink undertone and even after a greasy dinner i had it fade off and left a tint which is great the applicator of it s great for matte hippies as i can outline my lips with it the only concern i have was removing it but there s a specific eraser for it or else it can last for almost a day i ll definitely use it again and buy the other colours too
okay so this is an interesting formular for a matte lippies. i have tons of reason to love it!     - it’s quick drying and transferproof.   - it’s sticky before it’s dries but its normal for a matte lippies.   - it has a velvet felt when u first 

the maybelle super stay matte ink is perfect for my lips it does not smudge or transfer hence it stays for a long time without having to touch up additionally it does not crack or peel like other matte lipsticks may do and my lipstick do not feel dry
the maybelline super stay matte ink is perfect for my lips. it does not smudge or transfer hence it stays for a long time without having to touch up. additionally, it does not crack or peel like other matte lipsticks may do and my lipstick do not feel dry.
1 ['the', 'maybelline', 'super', 'stay', 'matte', 'ink', 'is', 'perfect', 'for', 'my', 'lips.', 'it', 'does', 'not', 'smudge', 'or', 'transfer', 'hence', 'it', 'stays', 'for', 'a', 'long', 'time', 'without', 'having', 'to', 'touch', 'up.', 'additionally,', 'it', 'does', 'not', 'crack', 'or', 'peel', 'like', 'other', 'matte', 'lipsticks', 'may', 'do', 'and', 'my', 'lipstick', 'do', 'not', 'feel', 'dry.']
2 the maybelline super stay matte ink is perfect for my lips. it does not smudge or t

the colour stayed on my lips for really around hours wow love the colour too
nice shade n its really awsum n supera long stay luv it totally
1 ['nice', 'shade', 'n', 'its', 'really', 'awsum', 'n', 'supera', 'long', 'stay', 'luv', 'it', 'totally']
2 nice shade n its really awsum n supera long stay luv it totally
nice shade n its really aksum n superb long stay luv it totally
worked like a charm, truly lives up to “superstay”.
1 ['worked', 'like', 'a', 'charm,', 'truly', 'lives', 'up', 'to', '“superstay”.']
2 worked like a charm, truly lives up to “superstay”.
worked like a charm truly lives up to superstar
long lasting，waterproof，kiss-proof，good smell，i really like it❤️
1 ['long', 'lasting，waterproof，kiss-proof，good', 'smell，i', 'really', 'like', 'it❤️']
2 long lasting，waterproof，kiss-proof，good smell，i really like it❤️
long lasting waterproof kiss proof good smell i really like it
it is worth to buy with this price and good condition. i like it
1 ['it', 'is', 'worth', 'to', 'buy', 'wit

ordered on apr and received on apr in the mailbox it can cover my dark spots it does crease in my eye area
exactly what i ordered. thanks!
1 ['exactly', 'what', 'i', 'ordered.', 'thanks!']
2 exactly what i ordered. thanks!
exactly what i ordered thanks
fast delivery. yet to try it.
1 ['fast', 'delivery.', 'yet', 'to', 'try', 'it.']
2 fast delivery. yet to try it.
fast delivery yet to try it
great product have repurchased more than 10 times
1 ['great', 'product', 'have', 'repurchased', 'more', 'than', '10', 'times']
2 great product have repurchased more than 10 times
great product have repurchased more than 10 times
very worth to buy during 11.11 sales. first time use n fall in love with it. it can blend well with my dry skin. will buy again .
1 ['very', 'worth', 'to', 'buy', 'during', '11.11', 'sales.', 'first', 'time', 'use', 'n', 'fall', 'in', 'love', 'with', 'it.', 'it', 'can', 'blend', 'well', 'with', 'my', 'dry', 'skin.', 'will', 'buy', 'again', '.']
2 very worth to buy during 11.

disappointed that the protective seal was already cut open it should be the case for new products
ordered on 24apr and received on 30apr in the mailbox. it can cover my dark spots. it doesnt crease in my eye area.
1 ['ordered', 'on', '24apr', 'and', 'received', 'on', '30apr', 'in', 'the', 'mailbox.', 'it', 'can', 'cover', 'my', 'dark', 'spots.', 'it', 'doesnt', 'crease', 'in', 'my', 'eye', 'area.']
2 ordered on 24apr and received on 30apr in the mailbox. it can cover my dark spots. it doesnt crease in my eye area.
ordered on apr and received on apr in the mailbox it can cover my dark spots it does crease in my eye area
fast delivery
1 ['fast', 'delivery']
2 fast delivery
fast delivery
wow super fast delivery!!!
1 ['wow', 'super', 'fast', 'delivery!!!']
2 wow super fast delivery!!!
wow super fast delivery
not as good as the age rewind for undereye concealer. probably works better on face.
1 ['not', 'as', 'good', 'as', 'the', 'age', 'rewind', 'for', 'undereye', 'concealer.', 'probably', 

In [18]:
#Store into a new data frame
splitted_data = {'Platform':platform_csv ,'Brand':brand_csv, 'Category': category_csv, 'Product Name ': product_name_csv, 'Price':prices_csv ,'Reviewer':reviewer_csv,'Review':review_csv, 'Review_splitted':review_splitted_csv, 'Product Purchase':product_variation_csv,'Ratings':rating_csv,'Date Of Review':date_review_csv,'Response': response_csv }
splitted_df = pd.DataFrame.from_dict(splitted_data)
splitted_df['Date Of Review'] = pd.to_datetime(splitted_df['Date Of Review'])
splitted_df.head()
splitted_df.info()

splitted_df.to_csv('AllData_Process_Topic.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8987 entries, 0 to 8986
Data columns (total 12 columns):
Platform            8987 non-null object
Brand               8987 non-null object
Category            8987 non-null object
Product Name        8987 non-null object
Price               8987 non-null float64
Reviewer            8965 non-null object
Review              8987 non-null object
Review_splitted     8987 non-null object
Product Purchase    7382 non-null object
Ratings             8965 non-null float64
Date Of Review      8965 non-null datetime64[ns]
Response            8965 non-null object
dtypes: datetime64[ns](1), float64(2), object(9)
memory usage: 842.6+ KB
